# Selekcja atrybutów - stepwise

### Algorytm (forward)

1. Zaczynamy od pustego modelu (0 zmiennych wybranych) i przyjmujemy, że wartość kryterium (np. BIC, AIC): C = Inf
2. Szukamy zmiennej, której dodanie do modelu najbardziej zmniejszy kryterium C. 
   * Jeśli wartość C jest mniejsza od poprzedniej wartości - dodajemy zmienną i kontynuujemy
   * Jeśli wartość C dla najlepszego znalezionego w tym kroku modelu nie spadła w stosunku do poprzedniego kroku,
     to nie dołączamy zmiennej i kończymy

## Zadanie 1

Zaimplementuj powyższy algorytm. 

## Zadanie 2

Przetestuj jego działanie na dobrze znanym zbiorze winequality-red.csv i porównaj wyniki z LASSO i Lasami Losowymi.


## Inne warianty

Powyższy algorytm można zmodyfikować, aby startował z pełneo modelu i w każdym kroku usuwał zmienną, tak aby minimalizować kryterium C (backward)

Można również startować z pustego albo pełneo modelu i w każdym kroku dodawać lub usuwać zmienną, zgodnie z powyższymi zasadami. Koniec nastąpi, gdy ani dodanie, a ni usunięcie zmiennej nie spowoduje spadku wartości kryterium.